In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException 

# Step 1: Create a session and load the page
'''
url1 = 'https://www.nba.com/stats/players/pullup?PerMode=Totals'
url2 = 'https://www.nba.com/stats/players/catch-shoot?PerMode=Totals'
url3 = 'https://www.nba.com/stats/players/defense-dash-lt6?PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/isolation?PerMode=Totals'
url5 = 'https://www.nba.com/stats/players/transition?PerMode=Totals&dir=D&sort=POSS'
'''
cs ='https://www.nba.com/stats/players/catch-shoot?PerMode=Totals'
pullup ='https://www.nba.com/stats/players/pullup?PerMode=Totals'

handoff = 'https://www.nba.com/stats/teams/hand-off?PerMode=Totals'
iso ='https://www.nba.com/stats/teams/isolation?PerMode=Totals'
trans ='https://www.nba.com/stats/teams/transition?PerMode=Totals'
bh='https://www.nba.com/stats/teams/ball-handler?PerMode=Totals'
rollman = 'https://www.nba.com/stats/teams/roll-man?PerMode=Totals'
postup = 'https://www.nba.com/stats/teams/playtype-post-up?PerMode=Totals'
spotup = 'https://www.nba.com/stats/teams/spot-up?PerMode=Totals'
cut = 'https://www.nba.com/stats/teams/cut?PerMode=Totals'
offscreen = 'https://www.nba.com/stats/teams/off-screen?PerMode=Totals'
putbacks = 'https://www.nba.com/stats/teams/putbacks?PerMode=Totals'
misc = 'https://www.nba.com/stats/teams/playtype-misc?PerMode=Totals'

#url_list = [url1,url2,url3,url4,url5]
url_list=[handoff,iso,trans,bh,rollman,postup,spotup,cut,offscreen,putbacks,misc]
def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True
#url_list = [cs,pullup]
def get_tables(url_list):
    data = []
    for url in url_list:
        print(url)
        driver = webdriver.Chrome()
        driver.get(url)

        # Wait for the page to fully load
        driver.implicitly_wait(8)
        '''if check_exists_by_xpath(driver, "//a[contains(text(),'>')]/preceding-sibling::a[1]"):
            number_of_pages = int(driver.find_element(By.XPATH, "//a[contains(text(),'>')]/preceding-sibling::a[1]").text)
            print(number_of_pages)'''
        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))
        #print(dfs)

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())
    
        driver.close()
        #return dfs
        df= dfs[-1]
        #drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        #df.columns = df.columns.droplevel()
        #df = df.drop(columns = drop)
        data.append(df)
    return data

In [2]:
#url_list = [url1]
frames = get_tables(url_list)

https://www.nba.com/stats/teams/hand-off?PerMode=Totals
https://www.nba.com/stats/teams/isolation?PerMode=Totals
https://www.nba.com/stats/teams/transition?PerMode=Totals
https://www.nba.com/stats/teams/ball-handler?PerMode=Totals
https://www.nba.com/stats/teams/roll-man?PerMode=Totals
https://www.nba.com/stats/teams/playtype-post-up?PerMode=Totals
https://www.nba.com/stats/teams/spot-up?PerMode=Totals
https://www.nba.com/stats/teams/cut?PerMode=Totals
https://www.nba.com/stats/teams/off-screen?PerMode=Totals
https://www.nba.com/stats/teams/putbacks?PerMode=Totals
https://www.nba.com/stats/teams/playtype-misc?PerMode=Totals


In [5]:
frames[10]

,TEAM,GP,POSS,FREQ%,PPP,PTS,FGM,FGA,FG%,EFG%,FTFREQ%,TOVFREQ%,SFFREQ%,AND ONEFREQ%,SCOREFREQ%,PERCENTILE
0,Utah Jazz,26,199,6.7,0.57,113,21,46,45.7,50.0,22.6,54.3,3.5,0.5,30.7,58.6
1,Detroit Pistons,25,184,6.5,0.58,106,17,49,34.7,39.0,23.9,50.0,5.4,0.5,32.1,72.4
2,Miami Heat,24,168,6.4,0.63,105,20,41,48.8,51.0,22.6,53.6,2.4,0.6,33.3,93.1
3,Portland Trail Blazers,24,170,6.5,0.61,103,13,33,39.4,47.0,28.2,52.9,1.8,0.6,33.5,86.2
4,Indiana Pacers,24,176,6.3,0.58,103,23,48,47.9,55.0,20.5,53.4,1.7,1.1,30.7,75.9
5,Brooklyn Nets,25,180,6.8,0.57,103,19,39,48.7,53.0,22.2,56.1,1.7,0.0,31.1,69.0
6,New York Knicks,24,142,5.1,0.66,94,19,46,41.3,47.0,23.9,45.8,4.9,2.1,35.2,100.0
7,Boston Celtics,25,160,5.9,0.59,94,14,31,45.2,55.0,24.4,56.9,2.5,0.6,30.6,79.3
8,Cleveland Cavaliers,24,152,5.8,0.61,93,15,38,39.5,41.0,25.7,50.0,3.3,0.7,33.6,89.7
9,Sacramento Kings,22,145,5.9,0.63,92,20,41,48.8,52.0,20.7,52.4,3.4,1.4,32.4,96.6


In [6]:
#terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']
terms = ['playtype/handoff.csv','playtype/iso.csv','playtype/trans.csv','playtype/bh.csv','playtype/rollman.csv','playtype/postup.csv','playtype/spotup.csv',
         'playtype/cut.csv','playtype/offscreen.csv','playtype/putback.csv','playtype/misc.csv']
for i in range(len(terms)):
    df = frames[i]
    df.to_csv(terms[i],index = False)
   